In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib notebook
%matplotlib notebook

In [2]:
# International segment monthly data from all carriers on all routes; there can be more than 1 entry for a month
all_2005 = pd.read_csv('./413215462_T_T100_SEGMENT_ALL_CARRIER.csv')  
# all_2005 = pd.read_csv('./113222843_T_T100_SEGMENT_ALL_CARRIER 3.csv')
columns = ['DEPARTURES_SCHEDULED', 'DEPARTURES_PERFORMED', 'SEATS',
           'PASSENGERS', 'DISTANCE', 'AIR_TIME',
           'UNIQUE_CARRIER', 'AIRLINE_ID', 'UNIQUE_CARRIER_NAME',
           'UNIQUE_CARRIER_ENTITY', 'REGION', 'CARRIER', 'CARRIER_NAME',
           'CARRIER_GROUP', 'CARRIER_GROUP_NEW', 'ORIGIN_AIRPORT_ID',
           'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN',
           'ORIGIN_CITY_NAME', 'ORIGIN_COUNTRY', 'ORIGIN_COUNTRY_NAME',
           'ORIGIN_WAC', 'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID',
           'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_COUNTRY',
           'DEST_COUNTRY_NAME', 'DEST_WAC', 'AIRCRAFT_GROUP', 'AIRCRAFT_TYPE',
           'AIRCRAFT_CONFIG', 'QUARTER', 'MONTH','DISTANCE_GROUP', 'CLASS']


all_2005 = all_2005[columns]

In [3]:
all_2005

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS,DISTANCE,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,...,DEST_COUNTRY,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,QUARTER,MONTH,DISTANCE_GROUP,CLASS
0,0.0,30.0,8946.0,8191.0,4801.0,0.0,BA,19540,British Airways Plc,9493E,...,US,United States,93,8,819,1,2,6,10,F
1,0.0,1.0,6.0,1.0,50.0,31.0,4Y,20324,Yute Air Aka Flight Alaska,06964,...,US,United States,1,0,35,1,3,8,1,F
2,0.0,19.0,0.0,0.0,347.0,1043.0,KR,20302,Kitty Hawk Aircargo,06688,...,US,United States,81,7,715,2,3,8,1,P
3,0.0,6.0,54.0,8.0,40.0,106.0,8E,20331,Bering Air Inc.,06992,...,US,United States,1,4,416,1,3,8,1,F
4,0.0,13.0,2236.0,2116.0,1506.0,0.0,AV,19537,Aerovias Nacl De Colombia,9327A,...,CO,Colombia,327,6,622,1,2,6,4,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442105,1467.0,1448.0,13032.0,8116.0,30.0,22589.0,9K,20253,Cape Air,07021,...,US,United States,13,1,125,1,3,9,1,F
442106,1468.0,1464.0,13176.0,8868.0,30.0,22838.0,9K,20253,Cape Air,07021,...,US,United States,13,1,125,1,3,7,1,F
442107,1486.0,1482.0,13338.0,9467.0,30.0,23119.0,9K,20253,Cape Air,07021,...,US,United States,13,1,125,1,3,8,1,F
442108,1554.0,1549.0,13941.0,9273.0,30.0,24164.0,9K,20253,Cape Air,07021,...,US,United States,13,1,125,1,3,8,1,F


In [4]:
# 255 WWLMINET network airport list
airport_list = pd.read_excel('./GHG and cost LC stages System Expansion.xlsx', sheet_name='Airport List Map')

# Parse based on the origin and destination
all_2005 = all_2005[all_2005['ORIGIN'].isin(airport_list.Code) & all_2005['DEST'].isin(airport_list.Code)]

In [5]:
# Filter out the all cargo carrier data based on carrier group new id
all_2005 = all_2005[all_2005['CARRIER_GROUP_NEW'] != 7]

In [6]:
# Filter out the freight configuration aircraft based on aircraft configuration id
all_2005 = all_2005[all_2005['AIRCRAFT_CONFIG'] != 2]

In [7]:
# Filter out seaplane based on aircraft configuration id
all_2005 = all_2005[all_2005['AIRCRAFT_CONFIG'] != 4]

In [8]:
# Filter out the all cargo scheduled service, unscheduled passenger service based on class id
all_2005 = all_2005[all_2005['CLASS'].isin(['F','A','C','E'])]

In [9]:
# Filter out routes by international/foreign carriers based on carrier group id and carrier group new id
all_2005 = all_2005[all_2005['CARRIER_GROUP_NEW'] != 0]
all_2005 = all_2005[all_2005['CARRIER_GROUP'] != 0]

In [10]:
# Keep only routes with demand for passenger service (non-zero passengers and seats) 
all_2005 = all_2005[(all_2005['PASSENGERS'] > 0) & (all_2005['SEATS'] > 0)]

In [11]:
# Filter out routes with 0 distance (or routes with origin = destination)
all_2005 = all_2005[all_2005['DISTANCE'] != 0]

In [12]:
# Aggregate the data together into the monthly performance by different airlines
# e.g. American Airlines has 1 entry for JFK-LAX on Jan, 1 on Feb, 1 on Apr, etc.

index_column = ['AIRLINE_ID', 'UNIQUE_CARRIER_NAME',
                'UNIQUE_CARRIER_ENTITY', 'REGION','CARRIER_NAME',
                'CARRIER_GROUP', 'CARRIER_GROUP_NEW', 'ORIGIN_AIRPORT_ID',
                'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN',
                'ORIGIN_CITY_NAME', 'ORIGIN_COUNTRY', 'ORIGIN_COUNTRY_NAME',
                'ORIGIN_WAC', 'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID',
                'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_COUNTRY',
                'DEST_COUNTRY_NAME', 'DEST_WAC', 'MONTH']
monthly_2005 = all_2005.groupby(index_column).agg({'DEPARTURES_SCHEDULED': 'sum', 
                                                 'DEPARTURES_PERFORMED': 'sum',  
                                                 'SEATS': 'sum', 
                                                 'PASSENGERS': 'sum',  
                                                 'DISTANCE': 'mean', 
                                                 'AIR_TIME': 'mean'})

monthly_2005.reset_index(inplace=True)

In [13]:
# Add directional route names (consider A-B != B-A)
monthly_2005['DIRROUTE'] = monthly_2005.apply(lambda x: x.ORIGIN+'-'+x.DEST, axis =1)

In [14]:
# Group by directional route names to combine all airlines into 1 representative airline 
# e.g. JFK-LAX has multiple entries (max 12), one for each month, and the demand shown is sum of all airlines

index_column = ['DIRROUTE', 'ORIGIN','ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME', 'MONTH']
monthly_2005 = monthly_2005.groupby(index_column).agg({'DEPARTURES_SCHEDULED': 'sum', 
                                                 'DEPARTURES_PERFORMED': 'sum',
                                                 'SEATS': 'sum',
                                                 'PASSENGERS': 'sum',
                                                 'DISTANCE': 'mean',
                                                 'AIR_TIME': 'mean'})
monthly_2005.reset_index(inplace=True)

In [15]:
# Aggregate the data together into the yearly performance by 1 combined airline
# e.g: There's 1 entry for JFK-LAX for 2005
# Removed UNIQUE_CARRIER and CARRIER to avoid filtering out an airline missing those entries

index_column = ['DIRROUTE', 'ORIGIN','ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME']
yearly_2005 = monthly_2005.groupby(index_column).agg({'DEPARTURES_SCHEDULED': 'sum', 
                                                 'DEPARTURES_PERFORMED': 'sum',  
                                                 'SEATS': 'sum', 
                                                 'PASSENGERS': 'sum',  
                                                 'DISTANCE': 'mean', 
                                                 'AIR_TIME': 'mean'})

yearly_2005.reset_index(inplace=True)

In [16]:
# A directional route must have at least 52 flights per year (1 flight per week) to qualify as "regular ops"
yearly_2005 = yearly_2005[yearly_2005['DEPARTURES_PERFORMED'] >= 52]
yearly_2005.to_csv('./yearly_demand_2005.csv')